In [40]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import torch
import re
from sklearn.metrics import balanced_accuracy_score, roc_auc_score,accuracy_score,precision_recall_fscore_support
from Constants import *
from Misc import *
from Utils import *
from Preprocessing import DTDataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
get_tt_split()

([5,
  6,
  8,
  11,
  13,
  14,
  15,
  16,
  17,
  18,
  21,
  23,
  24,
  26,
  27,
  28,
  32,
  33,
  37,
  38,
  39,
  40,
  41,
  42,
  48,
  49,
  50,
  51,
  53,
  55,
  56,
  57,
  60,
  64,
  65,
  67,
  69,
  71,
  74,
  75,
  78,
  79,
  80,
  81,
  82,
  87,
  88,
  91,
  94,
  96,
  99,
  103,
  109,
  116,
  119,
  120,
  121,
  125,
  148,
  150,
  153,
  178,
  181,
  183,
  185,
  186,
  188,
  191,
  192,
  193,
  196,
  197,
  198,
  200,
  201,
  203,
  204,
  205,
  206,
  207,
  210,
  212,
  213,
  214,
  216,
  218,
  219,
  220,
  221,
  222,
  223,
  225,
  226,
  229,
  230,
  231,
  232,
  233,
  234,
  235,
  237,
  238,
  239,
  240,
  241,
  243,
  244,
  246,
  247,
  248,
  249,
  251,
  252,
  253,
  255,
  256,
  257,
  258,
  259,
  260,
  261,
  262,
  263,
  265,
  266,
  269,
  270,
  273,
  275,
  276,
  277,
  278,
  280,
  281,
  282,
  283,
  285,
  289,
  2000,
  2002,
  2003,
  2004,
  2007,
  2008,
  2009,
  2010,
  2011,
  2012,
  2013,


In [42]:
data = DTDataset(use_smote=False)
data.processed_df[Const.timeseries_outcomes]

,OS (Calculated),Locoregional control (Time),FDM (months),time_to_event
id,,,,
3,6.033333,4.700000,6.033333,4.700000
5,7.333333,7.333333,7.333333,6.000000
6,7.466667,7.466667,7.466667,6.000000
7,7.800000,7.800000,7.800000,6.000000
8,8.066667,8.066667,8.066667,8.066667
...,...,...,...,...
10201,143.200000,143.200000,143.200000,143.200000
10202,144.366667,144.366667,144.366667,6.000000
10203,148.366667,148.366667,136.033333,136.033333


In [79]:
from DeepSurvivalModels import *
format_tte_outcomes(data,Const.timeseries_outcomes)[0][2].sum()

tensor(57)

In [9]:
xtr,xtst,ytr,ytst = transition_sample(3,data)
xtr.shape

torch.Size([389, 79])

In [89]:
from DeepSurvivalModels import *
test = DSM(xtr.shape[1],dist="Weibull")
test.fit_normalizer(xtr)
test = test.time_to_event(xtr)
test['predictions'][0].shape, test['5%'][0].shape, len(test['predictions']),len(test['5%'])

torch.Size([389, 3])
lmeans tensor([[4.6525e+00, 2.2027e+01, 1.2539e-01],
        [1.2371e-01, 7.5198e-03, 4.3508e-01],
        [2.4046e-01, 3.3495e+00, 9.8363e-02],
        ...,
        [2.4147e+00, 8.7211e+12, 1.2793e-01],
        [5.0411e+00, 2.4061e-01, 2.0342e+00],
        [9.3521e-01, 2.0853e-02, 1.4863e+00]], grad_fn=<StackBackward0>) logits tensor([[0.3334, 0.3332, 0.3334],
        [0.3334, 0.3332, 0.3334],
        [0.3332, 0.3333, 0.3335],
        ...,
        [0.3333, 0.3333, 0.3334],
        [0.3334, 0.3333, 0.3334],
        [0.3334, 0.3332, 0.3334]], grad_fn=<ExpBackward0>)
lmeans tensor([[7.7471e+00, 2.3359e+00, 3.0523e-04],
        [4.1998e+00, 4.9962e-02, 1.6790e+00],
        [1.7983e+03, 8.4847e+05, 1.2680e-02],
        ...,
        [9.1361e-13, 1.9831e+00, 2.1617e+00],
        [1.4376e-01, 2.7280e+00, 5.5041e-01],
        [1.3639e-02, 5.9155e-01,        inf]], grad_fn=<StackBackward0>) logits tensor([[0.3333, 0.3333, 0.3334],
        [0.3333, 0.3333, 0.3334],
        [

(torch.Size([389]), torch.Size([389]), 4, 4)

In [90]:
for i,ii,iii in zip(test['predictions'][0],test['5%'][0],test['95%'][0]):
    print(ii.item(),i.item(),iii.item())

0.6034491062164307 8.932765007019043 35158180.0
0.4635173976421356 0.1888038069009781 inf
0.5804634094238281 1.2293576002120972 490752.5
1.3455549478530884 129795463577600.0 nan
nan 4.438843727111816 nan
0.9761601090431213 1540.9583740234375 105375672041472.0
1.7269459962844849 13.587430953979492 1744724992.0
0.48957523703575134 2.4890122413635254 5.1904497146606445
0.6425434350967407 2.923947334289551 107.66325378417969
1.274430751800537 1.8634757995605469 22020.107421875
0.9920699000358582 1.823467493057251 198.64369201660156
1.833924651145935 5.53785514831543 68076249088.0
0.25845277309417725 0.7611932754516602 71.36328887939453
1.0794240236282349 670888185823232.0 37399.05078125
0.7486016750335693 501854109696.0 inf
0.286549836397171 2312.60498046875 54.118858337402344
0.7099966406822205 1.1158714294433594 159.79019165039062
nan 6187.50048828125 nan
0.01074493769556284 inf 224748.1875
0.6768674850463867 5.604600429534912 36036916.0
0.6802114248275757 1.1837917566299438 2.6948781286

In [ ]:
# test, loss, metrics = train_dsm(data,k=10,dist="Weibull")
# test.time_to_event(xtr)

In [76]:
test2, loss2, metrics2 = train_dsm(data,k=10,dist="LogNormal")
# test2.time_to_event(xtr)

best pretrain loss 7509.560365438118 epochs 4678
val loss 1.070396424875317
val metrics {'OS (Calculated)': {12: {'roc_score': 0.7220279720279721, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.5628342245989305, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.5177956371986223, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.5286259541984732, 'f1': 0.9424460431654677, 'matthews': 0.0}}, 'Locoregional control (Time)': {12: {'roc_score': 0.6987410071942446, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.6665076335877863, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.6539370078740158, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.6209677419354839, 'f1': 0.915129151291513, 'matthews': 0.0}}, 'FDM (months)': {12: {'roc_score': 0.6536643026004728, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.7290470723306544, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.70

val loss 0.8726490840455953
val metrics {'OS (Calculated)': {12: {'roc_score': 0.5506993006993006, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.5467914438502675, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.574052812858783, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.5190839694656488, 'f1': 0.9424460431654677, 'matthews': 0.0}}, 'Locoregional control (Time)': {12: {'roc_score': 0.6330935251798562, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.6369274809160306, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.6692913385826771, 'f1': 0.9328358208955224, 'matthews': 0.3192594883077374}, 48: {'roc_score': 0.6861851332398318, 'f1': 0.891566265060241, 'matthews': 0.2917299039538029}}, 'FDM (months)': {12: {'roc_score': 0.5401891252955082, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.708955223880597, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.6989664082687338, 'f1

val loss 0.8956656714890248
val metrics {'OS (Calculated)': {12: {'roc_score': 0.6538461538461539, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.6858288770053477, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.6687715269804821, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.5353053435114503, 'f1': 0.9386281588447654, 'matthews': -0.028923304887538078}}, 'Locoregional control (Time)': {12: {'roc_score': 0.6330935251798562, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.6631679389312977, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.6633858267716536, 'f1': 0.9288389513108615, 'matthews': 0.2839540830131564}, 48: {'roc_score': 0.6732117812061711, 'f1': 0.8941176470588236, 'matthews': 0.2102290458675726}}, 'FDM (months)': {12: {'roc_score': 0.5579196217494089, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.6859931113662456, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.

In [103]:
#model gridsearch
from DeepSurvivalModels import *
import pickle
layer_options = [
    [100,100],
    [100],
    [100,100,100]
]
results = []
best_results = {'criteria': 0}
main_file = '../resources/outcomeDSM.pt'
for layers in layer_options:
    for activation in ['Tanh','Sigmoid','ReLU6']:
        for dist in ['LogNormal']:
            for k in [6,9,12]:
                for elbo in [True]:
                    try:
                        args = {'k': k,'dist': dist,'layers':layers,'activation':activation,'elbo':elbo}
                        model, loss, metrics = train_dsm(data,dist=dist,k=k,layers=layers,activation=activation,elbo=elbo,verbose=1)
                        criteria = np.sum([v[12]['roc_score'] + v[48]['roc_score'] + v[48]['matthews'] for v in metrics.values()])
                        entry = {
                            'model': model,
                            'loss': loss,
                            'metrics': metrics,
                            'args': args,
                            'criteria': criteria
                        }
                        results.append(entry)
                        print('-----round' + str(len(results)) + '-------')
                        print(criteria,args)
                        if criteria > best_results['criteria']:
                            model.set_device('cpu')
                            torch.save(model,main_file)
                            best_results = {k:v for k,v in entry.items()}
                            print('++++++++_______new best______++++++++1+++')
                            print(best_results['criteria'],best_results['args'])
                            print('++++++++++++++++')
                        print()
                    except Exception as e:
                        print('^^^^^^^^^^^^^^^^^^^')
                        print('error with args',args)
                        print(e)
                        print('^^^^^^^^^^^^^^^^^^^^^^^^^^')

best_loss 0.7764825767395427
{'OS (Calculated)': {12: {'roc_score': 0.46853146853146854, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.6490641711229946, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.664179104477612, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.6173664122137404, 'f1': 0.9424460431654677, 'matthews': 0.0}}, 'Locoregional control (Time)': {12: {'roc_score': 0.599820143884892, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.5190839694656488, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.5661417322834645, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.5487377279102383, 'f1': 0.915129151291513, 'matthews': 0.0}}, 'FDM (months)': {12: {'roc_score': 0.458628841607565, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.5981630309988519, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.6007751937984496, 'f1': 0.9347826086956522, 'matthews': 0.0}, 4

best_loss 0.7718419265369095
{'OS (Calculated)': {12: {'roc_score': 0.5419580419580419, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.6604278074866311, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.6354764638346728, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.5939885496183206, 'f1': 0.9424460431654677, 'matthews': 0.0}}, 'Locoregional control (Time)': {12: {'roc_score': 0.6393884892086332, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.5233778625954199, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.539763779527559, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.5231416549789621, 'f1': 0.915129151291513, 'matthews': 0.0}}, 'FDM (months)': {12: {'roc_score': 0.5094562647754137, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.6205510907003444, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.61541774332472, 'f1': 0.9347826086956522, 'matthews': 0.0}, 48

best_loss 0.784585923272382
{'OS (Calculated)': {12: {'roc_score': 0.4020979020979021, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.6290106951871657, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.6463834672789897, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.5978053435114503, 'f1': 0.9424460431654677, 'matthews': 0.0}}, 'Locoregional control (Time)': {12: {'roc_score': 0.6393884892086331, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.5749045801526718, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.5921259842519685, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.5683730715287518, 'f1': 0.915129151291513, 'matthews': 0.0}}, 'FDM (months)': {12: {'roc_score': 0.557919621749409, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.6280137772675085, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.6304909560723514, 'f1': 0.9347826086956522, 'matthews': 0.0}, 4

best_loss 0.7767359673276618
{'OS (Calculated)': {12: {'roc_score': 0.4125874125874126, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.6196524064171123, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.6463834672789897, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.6269083969465649, 'f1': 0.9424460431654677, 'matthews': 0.0}}, 'Locoregional control (Time)': {12: {'roc_score': 0.5620503597122302, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.5047709923664122, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.5559055118110237, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.5466339410939692, 'f1': 0.915129151291513, 'matthews': 0.0}}, 'FDM (months)': {12: {'roc_score': 0.5, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.6567164179104477, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.6658053402239449, 'f1': 0.9347826086956522, 'matthews': 0.0}, 48: {'roc_scor

best_loss 0.7851091583754689
{'OS (Calculated)': {12: {'roc_score': 0.68006993006993, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.6036096256684491, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.5568312284730195, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.5267175572519084, 'f1': 0.9424460431654677, 'matthews': 0.0}}, 'Locoregional control (Time)': {12: {'roc_score': 0.5278776978417267, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.46898854961832065, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.4917322834645669, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.4838709677419355, 'f1': 0.915129151291513, 'matthews': 0.0}}, 'FDM (months)': {12: {'roc_score': 0.458628841607565, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.5338691159586682, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.5129198966408268, 'f1': 0.9347826086956522, 'matthews': 0.0}, 4

In [106]:
best_results

{'model': DSM(
   (act): Tanh()
   (shape): ParameterList(
       (0): Parameter containing: [torch.float32 of size 6]
       (1): Parameter containing: [torch.float32 of size 6]
       (2): Parameter containing: [torch.float32 of size 6]
       (3): Parameter containing: [torch.float32 of size 6]
   )
   (scale): ParameterList(
       (0): Parameter containing: [torch.float32 of size 6]
       (1): Parameter containing: [torch.float32 of size 6]
       (2): Parameter containing: [torch.float32 of size 6]
       (3): Parameter containing: [torch.float32 of size 6]
   )
   (gate): ModuleList(
     (0): Sequential(
       (0): Linear(in_features=103, out_features=6, bias=False)
     )
     (1): Sequential(
       (0): Linear(in_features=103, out_features=6, bias=False)
     )
     (2): Sequential(
       (0): Linear(in_features=103, out_features=6, bias=False)
     )
     (3): Sequential(
       (0): Linear(in_features=103, out_features=6, bias=False)
     )
   )
   (scaleg): ModuleList(

In [107]:
torch.save(best_results['model'],main_file)

In [100]:
from DeepSurvivalModels import *

bm= train_dsm(data,dist='Weibull',k=3,layers=[100],activation='Sigmoid',elbo=False)
bm[0].time_to_event(xtr,n_samples=1)[0]

best pretrain loss 10055.557072009233 epochs 3449
val loss 1.9759784693706217
val metrics {'OS (Calculated)': {12: {'roc_score': 0.30244755244755245, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.643048128342246, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.6963260619977039, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.6412213740458016, 'f1': 0.9424460431654677, 'matthews': 0.0}}, 'Locoregional control (Time)': {12: {'roc_score': 0.42266187050359716, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.3812022900763359, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.44645669291338586, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.4403927068723703, 'f1': 0.915129151291513, 'matthews': 0.0}}, 'FDM (months)': {12: {'roc_score': 0.3144208037825059, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.4161882893226177, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 

val loss 1.882563245254973
val metrics {'OS (Calculated)': {12: {'roc_score': 0.3898601398601399, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.6417112299465241, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.6618828932261767, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.6273854961832062, 'f1': 0.9424460431654677, 'matthews': 0.0}}, 'Locoregional control (Time)': {12: {'roc_score': 0.5503597122302158, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.49952290076335876, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.5299212598425197, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.5231416549789621, 'f1': 0.915129151291513, 'matthews': 0.0}}, 'FDM (months)': {12: {'roc_score': 0.41252955082742315, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.5470723306544202, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.6012058570198104, 'f1': 0.9347826086956522, 'matt

val loss 1.8704269596151097
val metrics {'OS (Calculated)': {12: {'roc_score': 0.4125874125874126, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.6116310160427807, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.6257175660160735, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.6135496183206107, 'f1': 0.9424460431654677, 'matthews': 0.0}}, 'Locoregional control (Time)': {12: {'roc_score': 0.5701438848920863, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.5152671755725191, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.5366141732283465, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.5406732117812061, 'f1': 0.915129151291513, 'matthews': 0.0}}, 'FDM (months)': {12: {'roc_score': 0.45744680851063824, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.5981630309988518, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.6261843238587426, 'f1': 0.9347826086956522, 'matt

val loss 1.8678770946042946
val metrics {'OS (Calculated)': {12: {'roc_score': 0.43006993006993005, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.5969251336898396, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.6159586681974741, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.6178435114503816, 'f1': 0.9424460431654677, 'matthews': 0.0}}, 'Locoregional control (Time)': {12: {'roc_score': 0.5629496402877698, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.5190839694656488, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.5338582677165354, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.5445301542776999, 'f1': 0.915129151291513, 'matthews': 0.0}}, 'FDM (months)': {12: {'roc_score': 0.4420803782505911, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.5665901262916189, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.6145564168819982, 'f1': 0.9347826086956522, 'matt

val loss 1.8764729776261273
val metrics {'OS (Calculated)': {12: {'roc_score': 0.45629370629370625, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.554144385026738, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.5763490241102182, 'f1': 0.9537366548042704, 'matthews': 0.0}, 48: {'roc_score': 0.6040076335877863, 'f1': 0.9347826086956521, 'matthews': -0.04104453503547053}}, 'Locoregional control (Time)': {12: {'roc_score': 0.572841726618705, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.5085877862595419, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.5066929133858267, 'f1': 0.927007299270073, 'matthews': 0.0}, 48: {'roc_score': 0.5291023842917252, 'f1': 0.915129151291513, 'matthews': 0.0}}, 'FDM (months)': {12: {'roc_score': 0.42316784869976365, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.5332950631458093, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: {'roc_score': 0.5934539190353143, 'f1': 0.9347826

val loss 1.881277568178945
val metrics {'OS (Calculated)': {12: {'roc_score': 0.486013986013986, 'f1': 0.9862068965517241, 'matthews': 0.0}, 24: {'roc_score': 0.5093582887700534, 'f1': 0.9611307420494699, 'matthews': 0.0}, 36: {'roc_score': 0.5304247990815154, 'f1': 0.9462365591397849, 'matthews': -0.03658055360021776}, 48: {'roc_score': 0.5658396946564885, 'f1': 0.9304029304029305, 'matthews': 0.05492246100579451}}, 'Locoregional control (Time)': {12: {'roc_score': 0.5773381294964028, 'f1': 0.9720279720279721, 'matthews': 0.0}, 24: {'roc_score': 0.4952290076335878, 'f1': 0.9424460431654677, 'matthews': 0.0}, 36: {'roc_score': 0.4968503937007874, 'f1': 0.9304029304029303, 'matthews': 0.20855011552365899}, 48: {'roc_score': 0.5189340813464236, 'f1': 0.9104477611940298, 'matthews': 0.07027129688546671}}, 'FDM (months)': {12: {'roc_score': 0.42316784869976365, 'f1': 0.9791666666666666, 'matthews': 0.0}, 24: {'roc_score': 0.4879448909299655, 'f1': 0.9537366548042704, 'matthews': 0.0}, 36: 

tensor([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, nan, inf, inf, nan, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, i

In [101]:
times = bm[0].time_to_event(xtr,n_samples=1)[0]
for i,time in enumerate(times):
    print(time.item(), bm[0](xtr,t=time.item())[0][0][i].item())
times

lmeans tensor([[       inf,        inf, 6.4336e+34],
        [1.0002e+00,        inf,        inf],
        [       inf, 1.0000e+00,        inf],
        ...,
        [       inf,        inf,        inf],
        [       inf,        inf,        inf],
        [       inf,        inf,        inf]], grad_fn=<StackBackward0>) logits tensor([[0.3333, 0.3329, 0.3338],
        [0.3332, 0.3333, 0.3336],
        [0.3334, 0.3329, 0.3337],
        ...,
        [0.3336, 0.3329, 0.3335],
        [0.3333, 0.3329, 0.3338],
        [0.3335, 0.3330, 0.3335]], grad_fn=<ExpBackward0>)
lmeans tensor([[inf, inf, inf],
        [1., inf, 1.],
        [inf, inf, inf],
        ...,
        [inf, 1., inf],
        [inf, inf, inf],
        [inf, 1., inf]], grad_fn=<StackBackward0>) logits tensor([[0.3338, 0.3340, 0.3322],
        [0.3340, 0.3341, 0.3319],
        [0.3336, 0.3341, 0.3323],
        ...,
        [0.3337, 0.3341, 0.3321],
        [0.3338, 0.3340, 0.3323],
        [0.3338, 0.3340, 0.3322]], grad_fn=<E

tensor([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, nan, inf, inf, nan, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, i

In [ ]:
import matplotlib.pyplot as plt
fig,axes = plt.subplots(2,2,figsize=(20,10))
i = 0
titles= Const.timeseries_outcomes
for row in range(axes.shape[0]):
    for col in range(axes.shape[1]):
        scurves = torch.stack(curves[i],axis=-1).detach().cpu().numpy()
        for ii in range(scurves.shape[0]):
            axes[row][col].plot(times,scurves[ii],color='black',alpha=.1,linewidth=2)
        axes[row][col].set_title(titles[i])
        axes[row][col].set_xlabel('Weeks')
        axes[row][col].set_ylabel('Survival %')
        i += 1